In [1]:
import os
import numpy as np
import cv2
import tqdm

In [2]:
image_root = "/fs/nexus-scratch/chuonghm/vm2m/data/I-HIM/HIM2K_M-HIM2K/images/natural"
alpha_root = "/fs/nexus-scratch/chuonghm/vm2m/data/I-HIM/HIM2K_M-HIM2K/alphas/natural"

pred_dirs = [
    "/fs/nexus-scratch/chuonghm/vm2m/output/HIM/i-him50k_nexus_231220_atten-128_final-c-128/vis/natural",
    "/fs/nexus-scratch/chuonghm/vm2m/output/HIM/i-him50k_nexus_231226_atten-128_final-c-64/vis/natural"
]
output_dir = "/fs/nexus-scratch/chuonghm/vm2m/output/HIM/compare_final-c"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
image_root = "/mnt/localssd/HIM2K/images/natural"
alpha_root = "/mnt/localssd/HIM2K/alphas/natural"

pred_dirs = [
    "/home/chuongh/vm2m/output/HIM/mgm_enc-dec-spconv_him_syn_short-576-512x512_bs256_20k_adamw_5.0e-5/vis_natural/alpha_pred/natural",
    "/home/chuongh/vm2m/output/HIM/mgm_enc-dec_multi-inst_him_bs12_2gpus_best0804/vis_natural/alpha_pred/natural",
    "/home/chuongh/vm2m/output/HIM/mgm_cvpr23/vis_natural/alpha_pred/natural"
]
output_dir = "/home/chuongh/vm2m/output/HIM/combine_him2k_0809"
os.makedirs(output_dir, exist_ok=True)

In [12]:
for image_name in tqdm.tqdm(os.listdir(image_root)):
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(alpha_root, image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * 3, 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    for alpha_name in alpha_names:
        alpha = cv2.imread(os.path.join(alpha_root, image_name.replace(".jpg", ""), alpha_name))
        alpha = cv2.resize(alpha, (instance_w, instance_h))
        if len(alpha.shape) == 1:
            alpha = np.expand_dims(alpha, axis=-1)
            alpha = np.concatenate([alpha, alpha, alpha], axis=-1)
        alpha[:2, :, 2] = 255
        alpha[-2:, :, 2] = 255
        alpha[:, :2, 2] = 255
        alpha[:, -2:, 2] = 255
        new_image[y: y + instance_h, x:x + instance_w] = alpha
        y += instance_h
    x += instance_w
    for pred_dir in pred_dirs:
        for i, alpha_name in enumerate(alpha_names):
            pred_name = os.path.join(pred_dir, image_name.replace(".jpg", f"/{alpha_name}"))
            pred = cv2.imread(pred_name)
            pred = cv2.resize(pred, (instance_w, instance_h))
            pred[:2, :, 2] = 255
            pred[-2:, :, 2] = 255
            pred[:, :2, 2] = 255
            pred[:, -2:, 2] = 255
            new_image[i * instance_h: (i + 1) * instance_h, x:x + instance_w] = pred
        x += instance_w
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [01:51<00:00,  2.86it/s]


In [10]:
alpha_names

['02.png', '04.png', '06.png', '09.png']

In [4]:
for image_name in tqdm.tqdm(os.listdir(image_root)):
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(alpha_root, image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * 4, 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    for alpha_name in alpha_names:
        alpha = cv2.imread(os.path.join(alpha_root, image_name.replace(".jpg", ""), alpha_name))
        alpha = cv2.resize(alpha, (instance_w, instance_h))
        if len(alpha.shape) == 1:
            alpha = np.expand_dims(alpha, axis=-1)
            alpha = np.concatenate([alpha, alpha, alpha], axis=-1)
        alpha[:2, :, 2] = 255
        alpha[-2:, :, 2] = 255
        alpha[:, :2, 2] = 255
        alpha[:, -2:, 2] = 255
        new_image[y: y + instance_h, x:x + instance_w] = alpha
        y += instance_h
    x += instance_w
    for pred_dir in pred_dirs:
        for i in range(len(alpha_names)):
            pred_name = os.path.join(pred_dir, image_name.replace(".jpg", f"_inst{i}.jpg"))
            pred = cv2.imread(pred_name)
            pred = cv2.resize(pred, (instance_w, instance_h))
            pred[:2, :, 2] = 255
            pred[-2:, :, 2] = 255
            pred[:, :2, 2] = 255
            pred[:, -2:, 2] = 255
            new_image[i * instance_h: (i + 1) * instance_h, x:x + instance_w] = pred
        x += instance_w
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)
    

  0%|          | 0/320 [00:00<?, ?it/s]100%|██████████| 320/320 [09:16<00:00,  1.74s/it]


In [13]:
cv2.imwrite('test_image.jpg', new_image)

True

In [3]:
image_root = "/mnt/localssd/HIM2K/images/natural"
alpha_root = "/mnt/localssd/HIM2K/alphas/natural"
alpha_inst_mat = "/home/chuongh/InstMatt/results/him2k_natural"
pred_dirs = [
    "/home/chuongh/vm2m/output/HIM/mgm_cvpr23/vis_natural/alpha_pred/natural",
    "/home/chuongh/vm2m/output/HIM/mgm_enc-dec-spconv_him_syn_short-576-512x512_bs256_20k_adamw_5.0e-5/vis_natural/alpha_pred/natural",
]
output_dir = "/home/chuongh/vm2m/output/HIM/combine_him2k_0814"
os.makedirs(output_dir, exist_ok=True)

In [3]:
for image_name in tqdm.tqdm(os.listdir(image_root)):
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(alpha_root, image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * 4, 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    for alpha_name in alpha_names:
        for i, root_dir in enumerate([alpha_root, alpha_inst_mat]):
            alpha = cv2.imread(os.path.join(root_dir, image_name.replace(".jpg", ""), alpha_name))
            alpha = cv2.resize(alpha, (instance_w, instance_h))
            if len(alpha.shape) == 1:
                alpha = np.expand_dims(alpha, axis=-1)
                alpha = np.concatenate([alpha, alpha, alpha], axis=-1)
            alpha[:2, :, 2] = 255
            alpha[-2:, :, 2] = 255
            alpha[:, :2, 2] = 255
            alpha[:, -2:, 2] = 255
            x_start = x + (instance_w * i)
            new_image[y: y + instance_h, x_start :x_start + instance_w] = alpha
        y += instance_h
    x += (instance_w * 2)
    for pred_dir in pred_dirs:
        for i in range(len(alpha_names)):
            pred_name = os.path.join(pred_dir, image_name.replace(".jpg", f"_inst{i}.jpg"))
            pred = cv2.imread(pred_name)
            pred = cv2.resize(pred, (instance_w, instance_h))
            pred[:2, :, 2] = 255
            pred[-2:, :, 2] = 255
            pred[:, :2, 2] = 255
            pred[:, -2:, 2] = 255
            new_image[i * instance_h: (i + 1) * instance_h, x:x + instance_w] = pred
        x += instance_w
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)

  0%|          | 0/320 [00:00<?, ?it/s]


NameError: name 'alpha_root' is not defined

In [6]:
# For masks = 0.5
image_root = "/mnt/localssd/HIM2K/images/natural"
pred_dirs = [
    "/mnt/localssd/HIM2K/alphas/natural",
    "given_masks",
    "/home/chuongh/vm2m/output/HIM/mgm_cvpr23/vis_natural/alpha_pred/natural",
    "/home/chuongh/InstMatt/results/him2k_natural",
    "/home/chuongh/vm2m/output/HIM/mgm_enc-dec-spconv_atten-loss_him_syn_short-576-512x512_bs128_20k_adamw_5.0e-5/vis_natural/alpha_pred/natural"
]
output_dir = "/home/chuongh/vm2m/output/HIM/combine_him2k_0816"
os.makedirs(output_dir, exist_ok=True)

for image_name in tqdm.tqdm(os.listdir(image_root)):
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(pred_dirs[0], image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * len(pred_dirs), 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    
    for inst_i, alpha_name in enumerate(alpha_names):
        alpha_gt = None
        for i, pred_dir in enumerate(pred_dirs):
            mask = None
            if pred_dir == "given_masks":
                # Generate masks based on alpha
                mask = (alpha_gt > 0.5).astype('uint8') * 255
                mask = cv2.resize(alpha_gt, (alpha_gt.shape[1] // 8, alpha_gt.shape[0] // 8), interpolation=cv2.INTER_NEAREST)
            elif os.path.exists(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name)):
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name))
            else:
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", f"_inst{inst_i}.jpg")))
            if i == 0:
                alpha_gt = mask
            mask = cv2.resize(mask, (instance_w, instance_h))
            if len(mask.shape) == 1:
                mask = np.expand_dims(mask, axis=-1)
                mask = np.concatenate([mask, mask, mask], axis=-1)
            mask[:2, :, 2] = 255
            mask[-2:, :, 2] = 255
            mask[:, :2, 2] = 255
            mask[:, -2:, 2] = 255
            new_image[y: y + instance_h, x + (instance_w * i):x + (instance_w * (i + 1))] = mask
        y += instance_h
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)

  0%|          | 0/320 [00:00<?, ?it/s]100%|██████████| 320/320 [04:33<00:00,  1.17it/s]


In [8]:
# For masks = 0.5
image_root = "/mnt/localssd/HIM2K/images/natural"
pred_dirs = [
    "/mnt/localssd/HIM2K/alphas/natural",
    "/mnt/localssd/HIM2K/masks_matched/natural",
    "/home/chuongh/vm2m/output/HIM/mgm_cvpr23/vis_natural_random/alpha_pred/natural",
    "/home/chuongh/InstMatt/results/him2k_natural_random",
    "/home/chuongh/vm2m/output/HIM/mgm_enc-dec-spconv_atten-loss_him_syn_short-576-512x512_bs128_20k_adamw_5.0e-5/vis_natural_random/alpha_pred/natural"
]
output_dir = "/home/chuongh/vm2m/output/HIM/combine_him2k_random_0816"
os.makedirs(output_dir, exist_ok=True)

for image_name in tqdm.tqdm(os.listdir(image_root)):
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(pred_dirs[1], image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * len(pred_dirs), 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    
    for inst_i, alpha_name in enumerate(alpha_names):
        alpha_gt = None
        for i, pred_dir in enumerate(pred_dirs):
            mask = None
            if os.path.exists(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name)):
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name))
            else:
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", f"_inst{inst_i}.jpg")))
            mask = cv2.resize(mask, (instance_w, instance_h))
            if len(mask.shape) == 1:
                mask = np.expand_dims(mask, axis=-1)
                mask = np.concatenate([mask, mask, mask], axis=-1)
            mask[:2, :, 2] = 255
            mask[-2:, :, 2] = 255
            mask[:, :2, 2] = 255
            mask[:, -2:, 2] = 255
            new_image[y: y + instance_h, x + (instance_w * i):x + (instance_w * (i + 1))] = mask
        y += instance_h
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)

100%|██████████| 320/320 [04:42<00:00,  1.13it/s]
Bad pipe message: %s [b'n\xeb\xae\xd1\x89,j\x0f']
Bad pipe message: %s [b"\xa3Y\x9d9P\x1cW\x0c R\xef`\x07\xe7\xbd\x86h\x82Ka\xd3B\xaf\xd592\x92Eh\xa3J\xc9.\xbd\xdb\x8en\xbdH\x91\xdc\x00>\x13\x02\x13\x03\x13\x01\xc0,\xc00\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0+\xc0/\x00\x9e\xc0$\xc0(\x00k\xc0#\xc0'\x00g\xc0\n\xc0\x14\x009\xc0\t\xc0\x13\x003\x00\x9d\x00\x9c\x00=\x00<\x005\x00/\x00\xff\x01\x00\x01u\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t", b'\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\x02\x03', b'\x02\x01', b'\x02\x02']
Bad pipe message: %s [b'\x05\x02\x06']
Bad pipe message: %s [b'+']


In [ ]:
# For masks = 0.5
image_root = "/mnt/localssd/HIM2K/images/natural"
pred_dirs = [
    "/mnt/localssd/HIM2K/alphas/natural",
    "/home/chuongh/sensei-fs-symlink/users/chuongh/vm2m/output/HIM/ours_1031_single-stage_ft-details/debug/alpha_pred/natural",
    "/home/chuongh/sensei-fs-symlink/users/chuongh/vm2m/output/HIM/mgm_cvpr23_stacked_multi-inst_him_bs12_0920/vis/alpha_pred/natural"
]
output_dir = "/home/chuongh/vm2m/output/HIM/combine_him2k_1102"
os.makedirs(output_dir, exist_ok=True)

target_files = set([
    "unsplash_middle_saeed-karimi-JrrWC7Qcmhs-unsplash",
    "google_easy_c3668dda4d46436097b6c5d58153e7de",
    "google_middle_95e6da3a2f164a80b68abdd8058aa696",
    "google_middle_62604586fc08499ca4d346900c338bc8",
    "google_middle_44440b07b2d5459ea2053cd3ee9f9406",
    "google_middle_2db0e2572a654f55947d88d411c3cb61",
    "celebrity_middle_d3874137945e41319d93448910b8bcdd",
    "celebrity_easy_e129df9159a64841b62c588ddc5730c9",
    "celebrity_middle_1650c9fd73ae4313bae6a598a7d7072e",
    "celebrity_middle_8f47dc49975f4fc484224fdec092253f",
    "celebrity_easy_c0c60daf04cf41e3bb5752d932944dd9",
    "celebrity_middle_2b9db99c95e54d918f2a0795515ef271",
    "celebrity_easy_b71d8703a11240a2aba6babc1193c2e4",
    "Pexels_middle_pexels-photo-5896435".
    "Pexels_easy_pexels-photo-5618157"
])

# for image_name in tqdm.tqdm(os.listdir(image_root)):
for image_name in tqdm.tqdm(target_files):
    image_name = image_name + ".jpg"
    image = cv2.imread(os.path.join(image_root, image_name))
    h, w, _ = image.shape
    w = int(2000 / h * w)
    image = cv2.resize(image, (w, 2000))

    # Load all alphas
    alpha_names = sorted(os.listdir(os.path.join(pred_dirs[1], image_name.replace(".jpg", ""))))
    instance_h = 2000 // len(alpha_names)
    instance_w = int(w * instance_h / 2000)

    new_image = np.zeros((2000, w + instance_w * len(pred_dirs), 3), dtype=np.uint8)
    new_image[:, :w] = image
    x = w
    y = 0
    
    for inst_i, alpha_name in enumerate(alpha_names):
        alpha_gt = None
        for i, pred_dir in enumerate(pred_dirs):
            mask = None
            if os.path.exists(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name)):
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", ""), alpha_name))
            else:
                mask = cv2.imread(os.path.join(pred_dir, image_name.replace(".jpg", f"_inst{inst_i}.jpg")))
            mask = cv2.resize(mask, (instance_w, instance_h))
            if len(mask.shape) == 1:
                mask = np.expand_dims(mask, axis=-1)
                mask = np.concatenate([mask, mask, mask], axis=-1)
            mask[:2, :, 2] = 255
            mask[-2:, :, 2] = 255
            mask[:, :2, 2] = 255
            mask[:, -2:, 2] = 255
            new_image[y: y + instance_h, x + (instance_w * i):x + (instance_w * (i + 1))] = mask
        y += instance_h
    cv2.imwrite(os.path.join(output_dir, image_name), new_image)

  0%|          | 0/320 [00:00<?, ?it/s]100%|██████████| 320/320 [03:19<00:00,  1.61it/s]
